In [ ]:
#pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.0 MB/s eta 0:00:00


In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 34.1 MB/s eta 0:00:00


In [ ]:
import os

# Specify the directory where the images are stored
directory = '/content/images'

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the file has a .png extension
    if filename.endswith('.png'):
        # Construct full file path
        file_path = os.path.join(directory, filename)

        try:
            # Remove the file
            os.remove(file_path)
            print(f"Deleted {filename}")
        except Exception as e:
            print(f"Error deleting {filename}: {e}")


Deleted pic18.png
Deleted pic19.png
Deleted Picture5.png
Deleted Picture12.png
Deleted pic10.png
Deleted pic4.png
Deleted pic14.png
Deleted pic15.png
Deleted pic3.png
Deleted pic5.png
Deleted Picture7.png
Deleted pic17.png
Deleted pic7.png
Deleted pic6.png
Deleted Picture10.png
Deleted Picture17.png
Deleted pic16.png
Deleted Picture15.png
Deleted pic11.png
Deleted pic2.png
Deleted pic1.png
Deleted pic21.png
Deleted Picture8.png
Deleted pic20.png
Deleted Picture16.png
Deleted pic22.png
Deleted Picture22.png
Deleted pic9.png
Deleted Picture14.png
Deleted Picture11.png
Deleted Picture9.png
Deleted Picture4.png
Deleted Picture19.png
Deleted Picture3.png
Deleted Picture6.png
Deleted Picture18.png
Deleted pic13.png
Deleted Picture21.png
Deleted pic8.png
Deleted pic12.png
Deleted Picture13.png
Deleted Picture20.png


ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


Saved symbol as /content/extracted_symbols/symbol_page_1_362.png


In [ ]:
import cv2
import os
import random
import json
import numpy as np
from datetime import datetime

# Your provided icon dimensions
icon_dimensions = {
    "pic18.png": (99, 36),
    "pic19.png": (81, 40),
    "pic10.png": (162, 56),
    "pic4.png": (191, 64),
    "pic14.png": (45, 40),
    "pic15.png": (71, 69),
    "pic3.png": (88, 42),
    "pic5.png": (221, 77),
    "pic17.png": (35, 31),
    "pic7.png": (66, 53),
    "pic6.png": (60, 52),
    "pic16.png": (50, 39),
    "pic11.png": (51, 68),
    "pic2.png": (200, 29),
    "pic1.png": (155, 91),
    "pic21.png": (55, 30),
    "pic20.png": (61, 34),
    "pic22.png": (53, 29),
    "pic9.png": (174, 54),
    "pic13.png": (46, 41),
    "pic8.png": (68, 58),
    "pic12.png": (45, 41)  # Included pic12.png
}

# Function to rotate image (just for bounding box calculation)
def rotate_image_for_bbox(image, angle):
    # This only rotates for bounding box calculation to avoid cropping
    rotated = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    cos = np.abs(rotated[0, 0])
    sin = np.abs(rotated[0, 1])
    new_w = int((image.shape[0] * sin) + (image.shape[1] * cos))
    new_h = int((image.shape[0] * cos) + (image.shape[1] * sin))
    return new_w, new_h

# Function to place images randomly on a large background with random rotation
def generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=500, large_bg_size=(1500, 1500), padding=20):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # COCO JSON format structure
    coco_output = {
        "info": {
            "year": "2024",
            "version": "1",
            "description": "Exported from roboflow.com",
            "contributor": "",
            "url": "https://public.roboflow.com/object-detection/undefined",
            "date_created": datetime.now().isoformat()
        },
        "licenses": [{
            "id": 1,
            "url": "https://creativecommons.org/licenses/by/4.0/",
            "name": "CC BY 4.0"
        }],
        "categories": [],  # This will now be filled based on icon names
        "images": [],
        "annotations": []
    }

    # Add categories based on the icon names (e.g., pic18 -> pic18 as category name)
    category_id_map = {}
    category_id = 0
    for icon_name in icon_dimensions.keys():
        category_name = icon_name.split('.')[0]  # Get category name from file name (e.g., pic18)
        coco_output['categories'].append({
            "id": category_id,
            "name": category_name,
            "supercategory": "icon"
        })
        category_id_map[icon_name] = category_id
        category_id += 1

    annotation_id = 0
    for i in range(image_count):
        # Create a larger white background to avoid clipping during rotation
        background = np.ones((large_bg_size[1], large_bg_size[0], 3), dtype=np.uint8) * 255

        # Image metadata for COCO JSON
        image_id = i
        image_file_name = f"generated_image_{i+1}.png"
        coco_output['images'].append({
            "id": image_id,
            "license": 1,
            "file_name": image_file_name,
            "height": large_bg_size[1],
            "width": large_bg_size[0],
            "date_captured": datetime.now().isoformat()
        })

        # Randomly pick one image
        img_file = random.choice(image_files)
        img = cv2.imread(os.path.join(folder_path, img_file))

        # Use the dimensions from the dictionary for the icon
        img_width, img_height = icon_dimensions[img_file]

        # Resize image to its original dimensions (don't modify the size)
        img = cv2.resize(img, (img_width, img_height))

        # Apply random rotation (for placing, we don't rotate)
        angle = random.randint(0, 360)

        # Calculate bounding box size after rotation
        new_w, new_h = rotate_image_for_bbox(img, angle)

        # Ensure the image is fully within the background by placing it in the center
        x1 = (large_bg_size[0] - new_w) // 2
        y1 = (large_bg_size[1] - new_h) // 2

        # Place image on the large background without rotating it (yet)
        background[y1:y1 + img_height, x1:x1 + img_width] = img

        # Calculate the center of the icon
        center_x = x1 + img_width // 2
        center_y = y1 + img_height // 2

        # Create a bounding box with padding based on the actual icon size
        bbox_x1 = max(0, center_x - (img_width // 2) - padding)
        bbox_y1 = max(0, center_y - (img_height // 2) - padding)
        bbox_width = img_width + 2 * padding
        bbox_height = img_height + 2 * padding

        # Add annotation in COCO format with the dynamically sized bounding box
        bbox = [bbox_x1, bbox_y1, bbox_width, bbox_height]

        # Assign the correct category ID based on the image file
        category_id = category_id_map[img_file]

        coco_output['annotations'].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,  # Use correct category ID
            "bbox": bbox,
            "area": bbox_width * bbox_height,
            "segmentation": [],
            "iscrowd": 0
        })
        annotation_id += 1

        # Save the background as an image
        output_image_path = os.path.join(output_folder, image_file_name)
        cv2.imwrite(output_image_path, background)
        print(f"Saved {output_image_path}")

    # Save the COCO-style annotations in JSON format
    with open(json_output, "w") as f:
        json.dump(coco_output, f, indent=4)
    print(f"Saved annotations to {json_output}")

# Example usage
folder_path = '/content/images'  # Path to your folder with individual images
output_folder = '/content/path_to_output_images/'  # Where you want to save generated images
json_output = '/content/annotations_coco.json'  # Where you want to save the COCO JSON file

os.makedirs(output_folder, exist_ok=True)

# Generate 500 images with random placements and COCO-style annotations
generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=500)

# Path to the output folder with the generated images and JSON file
output_folder = '/content/path_to_output_images'  # Replace with your actual output folder path
zip_filename = '/content/output_archive.zip'  # The desired path for the zip file

# Create a zip file from the output folder
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_folder)

print(f"Zipped folder saved as: {zip_filename}")


Saved /content/path_to_output_images/generated_image_1.png
Saved /content/path_to_output_images/generated_image_2.png
Saved /content/path_to_output_images/generated_image_3.png
Saved /content/path_to_output_images/generated_image_4.png
Saved /content/path_to_output_images/generated_image_5.png
Saved /content/path_to_output_images/generated_image_6.png
Saved /content/path_to_output_images/generated_image_7.png
Saved /content/path_to_output_images/generated_image_8.png
Saved /content/path_to_output_images/generated_image_9.png
Saved /content/path_to_output_images/generated_image_10.png
Saved /content/path_to_output_images/generated_image_11.png
Saved /content/path_to_output_images/generated_image_12.png
Saved /content/path_to_output_images/generated_image_13.png
Saved /content/path_to_output_images/generated_image_14.png
Saved /content/path_to_output_images/generated_image_15.png
Saved /content/path_to_output_images/generated_image_16.png
Saved /content/path_to_output_images/generated_im

In [ ]:
import shutil
import os

# Path to the output folder with the generated images and JSON file
output_folder = '/content/path_to_output_images'  # Replace with your actual output folder path
zip_filename = '/content/output_archive.zip'  # The desired path for the zip file

# Create a zip file from the output folder
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_folder)

print(f"Zipped folder saved as: {zip_filename}")

Zipped folder saved as: /content/output_archive.zip


Code with dictionary removed


In [ ]:
import cv2
import os
import random
import json
import numpy as np
from datetime import datetime

# Function to rotate the image randomly
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Rotate the image by a random angle
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated

# Function to place images randomly on a white background with random rotation
def generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=500, img_size=(250, 250), bg_size=(1000, 800)):  # Updated sizes
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # COCO JSON format structure
    coco_output = {
        "info": {
            "year": "2024",
            "version": "1",
            "description": "Exported from roboflow.com",
            "contributor": "",
            "url": "https://public.roboflow.com/object-detection/undefined",
            "date_created": datetime.now().isoformat()
        },
        "licenses": [{
            "id": 1,
            "url": "https://creativecommons.org/licenses/by/4.0/",
            "name": "CC BY 4.0"
        }],
        "categories": [
            {"id": 0, "name": "intern", "supercategory": "none"},
            {"id": 1, "name": "cir_in_rectangle", "supercategory": "intern"},
            {"id": 2, "name": "console", "supercategory": "intern"},
            {"id": 3, "name": "fan", "supercategory": "intern"},
            {"id": 4, "name": "pine_circle", "supercategory": "intern"},
            {"id": 5, "name": "poin_console", "supercategory": "intern"},
            {"id": 6, "name": "semicircle", "supercategory": "intern"}
        ],
        "images": [],
        "annotations": []
    }

    annotation_id = 0
    for i in range(image_count):
        # Create a white background
        background = np.ones((bg_size[1], bg_size[0], 3), dtype=np.uint8) * 255

        # Image metadata for COCO JSON
        image_id = i
        image_file_name = f"generated_image_{i+1}.png"
        coco_output['images'].append({
            "id": image_id,
            "license": 1,
            "file_name": image_file_name,
            "height": bg_size[1],
            "width": bg_size[0],
            "date_captured": datetime.now().isoformat()
        })

        if (i + 1) % 4 == 0:
            # For every 4th image, place two random images
            img_file1 = random.choice(image_files)
            img_file2 = random.choice(image_files)

            img1 = cv2.imread(os.path.join(folder_path, img_file1))
            img2 = cv2.imread(os.path.join(folder_path, img_file2))

            # Resize images to the common size 250x250
            img1 = cv2.resize(img1, img_size)
            img2 = cv2.resize(img2, img_size)

            # Apply random rotation to both images
            angle1 = random.randint(0, 360)
            angle2 = random.randint(0, 360)
            img1 = rotate_image(img1, angle1)
            img2 = rotate_image(img2, angle2)

            # Calculate the center and adjust the position so that the bounding box has the same center
            x1_1 = random.randint(0, bg_size[0] - img_size[0])
            y1_1 = random.randint(0, bg_size[1] - img_size[1])
            x1_2 = random.randint(0, bg_size[0] - img_size[0])
            y1_2 = random.randint(0, bg_size[1] - img_size[1])

            # Place both images on the background
            background[y1_1:y1_1 + img_size[1], x1_1:x1_1 + img_size[0]] = img1
            background[y1_2:y1_2 + img_size[1], x1_2:x1_2 + img_size[0]] = img2

            # Calculate bounding box centered on the icon's center
            bbox1 = [x1_1, y1_1, img_size[0], img_size[1]]
            bbox2 = [x1_2, y1_2, img_size[0], img_size[1]]

            # Add annotations for both images in COCO format
            coco_output['annotations'].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": random.randint(1, 6),  # Random category ID
                "bbox": bbox1,
                "area": img_size[0] * img_size[1],
                "segmentation": [],
                "iscrowd": 0
            })
            annotation_id += 1

            coco_output['annotations'].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": random.randint(1, 6),  # Random category ID
                "bbox": bbox2,
                "area": img_size[0] * img_size[1],
                "segmentation": [],
                "iscrowd": 0
            })
            annotation_id += 1
        else:
            # Randomly pick one image for other cases
            img_file = random.choice(image_files)
            img = cv2.imread(os.path.join(folder_path, img_file))

            # Resize image to the common size 250x250
            img = cv2.resize(img, img_size)

            # Apply random rotation
            angle = random.randint(0, 360)
            img = rotate_image(img, angle)

            # Calculate the center and adjust the position so that the bounding box has the same center
            x1 = random.randint(0, bg_size[0] - img_size[0])
            y1 = random.randint(0, bg_size[1] - img_size[1])

            # Place image on the background
            background[y1:y1 + img_size[1], x1:x1 + img_size[0]] = img

            # Calculate bounding box centered on the icon's center
            bbox = [x1, y1, img_size[0], img_size[1]]

            # Add annotation in COCO format
            coco_output['annotations'].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": random.randint(1, 6),  # Random category ID
                "bbox": bbox,
                "area": img_size[0] * img_size[1],
                "segmentation": [],
                "iscrowd": 0
            })
            annotation_id += 1

        # Save the background as an image
        output_image_path = os.path.join(output_folder, image_file_name)
        cv2.imwrite(output_image_path, background)
        print(f"Saved {output_image_path}")

    # Save the COCO-style annotations in JSON format
    with open(json_output, "w") as f:
        json.dump(coco_output, f, indent=4)
    print(f"Saved annotations to {json_output}")

# Example usage
folder_path = '/content/images'  # Path to your folder with individual images
output_folder = '/content/path_to_output_images/'  # Where you want to save generated images
json_output = '/content/annotations_coco.json'  # Where you want to save the COCO JSON file

os.makedirs(output_folder, exist_ok=True)

# Generate 500 images with random placements and COCO-style annotations
generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=500)


Favourite Code

In [ ]:
import cv2
import os
import random
import json
import numpy as np
from datetime import datetime

icon_dimensions = {
    "pic18.png": (99, 36),
    "pic19.png": (81, 40),
    "pic10.png": (162, 56),
    "pic4.png": (191, 64),
    "pic14.png": (45, 40),
    "pic15.png": (71, 69),
    "pic3.png": (88, 42),
    "pic5.png": (221, 77),
    "pic17.png": (35, 31),
    "pic7.png": (66, 53),
    "pic6.png": (60, 52),
    "pic16.png": (50, 39),
    "pic11.png": (51, 68),
    "pic2.png": (200, 29),
    "pic1.png": (155, 91),
    "pic21.png": (55, 30),
    "pic20.png": (61, 34),
    "pic22.png": (53, 29),
    "pic9.png": (174, 54),
    "pic13.png": (46, 41),
    "pic8.png": (68, 58),
    "pic12.png": (45, 41)  # Included pic12.png
}


# Function to rotate the image randomly
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Rotate the image by a random angle
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated

# Function to place images randomly on a white background with random rotation
def generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=500, bg_size=(1000, 800), padding=20):
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # COCO JSON format structure
    coco_output = {
        "info": {
            "year": "2024",
            "version": "1",
            "description": "Exported from roboflow.com",
            "contributor": "",
            "url": "https://public.roboflow.com/object-detection/undefined",
            "date_created": datetime.now().isoformat()
        },
        "licenses": [{
            "id": 1,
            "url": "https://creativecommons.org/licenses/by/4.0/",
            "name": "CC BY 4.0"
        }],
        "categories": [],  # This will now be filled based on icon names
        "images": [],
        "annotations": []
    }

    # Add categories based on the icon names (e.g., pic18 -> pic18 as category name)
    category_id_map = {}
    category_id = 0
    for icon_name in icon_dimensions.keys():
        category_name = icon_name.split('.')[0]  # Get category name from file name (e.g., pic18)
        coco_output['categories'].append({
            "id": category_id,
            "name": category_name,
            "supercategory": "icon"
        })
        category_id_map[icon_name] = category_id
        category_id += 1

    annotation_id = 0
    for i in range(image_count):
        # Create a white background
        background = np.ones((bg_size[1], bg_size[0], 3), dtype=np.uint8) * 255

        # Image metadata for COCO JSON
        image_id = i
        image_file_name = f"generated_image_{i+1}.png"
        coco_output['images'].append({
            "id": image_id,
            "license": 1,
            "file_name": image_file_name,
            "height": bg_size[1],
            "width": bg_size[0],
            "date_captured": datetime.now().isoformat()
        })

        # Randomly pick one image
        img_file = random.choice(image_files)
        img = cv2.imread(os.path.join(folder_path, img_file))

        # Use the dimensions from the dictionary for the icon
        img_width, img_height = icon_dimensions[img_file]

        # Resize image to its original dimensions (don't modify the size)
        img = cv2.resize(img, (img_width, img_height))

        # Apply random rotation
        angle = random.randint(0, 360)
        img = rotate_image(img, angle)

        # Ensure the image is fully within the background
        x1 = random.randint(0, bg_size[0] - img_width)
        y1 = random.randint(0, bg_size[1] - img_height)

        # Place image on the background
        background[y1:y1 + img_height, x1:x1 + img_width] = img

        # Calculate the center of the icon
        center_x = x1 + img_width // 2
        center_y = y1 + img_height // 2

        # Create a bounding box with padding based on the actual icon size
        bbox_x1 = max(0, center_x - (img_width // 2) - padding)
        bbox_y1 = max(0, center_y - (img_height // 2) - padding)
        bbox_width = img_width + 2 * padding
        bbox_height = img_height + 2 * padding

        # Add annotation in COCO format with the dynamically sized bounding box
        bbox = [bbox_x1, bbox_y1, bbox_width, bbox_height]

        # Assign the correct category ID based on the image file
        category_id = category_id_map[img_file]

        coco_output['annotations'].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,  # Use correct category ID
            "bbox": bbox,
            "area": bbox_width * bbox_height,
            "segmentation": [],
            "iscrowd": 0
        })
        annotation_id += 1

        # Save the background as an image
        output_image_path = os.path.join(output_folder, image_file_name)
        cv2.imwrite(output_image_path, background)
        print(f"Saved {output_image_path}")

    # Save the COCO-style annotations in JSON format
    with open(json_output, "w") as f:
        json.dump(coco_output, f, indent=4)
    print(f"Saved annotations to {json_output}")

# Example usage
folder_path = '/content/images'  # Path to your folder with individual images
output_folder = '/content/path_to_output_images/'  # Where you want to save generated images
json_output = '/content/annotations_coco.json'  # Where you want to save the COCO JSON file

os.makedirs(output_folder, exist_ok=True)

# Generate 500 images with random placements and COCO-style annotations
generate_image_with_bounding_boxes(folder_path, output_folder, json_output, image_count=3000)

# Path to the output folder with the generated images and JSON file
output_folder = '/content/path_to_output_images'  # Replace with your actual output folder path
zip_filename = '/content/output_archive.zip'  # The desired path for the zip file

# Create a zip file from the output folder
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_folder)

print(f"Zipped folder saved as: {zip_filename}")


Saved /content/path_to_output_images/generated_image_1.png
Saved /content/path_to_output_images/generated_image_2.png
Saved /content/path_to_output_images/generated_image_3.png
Saved /content/path_to_output_images/generated_image_4.png
Saved /content/path_to_output_images/generated_image_5.png
Saved /content/path_to_output_images/generated_image_6.png
Saved /content/path_to_output_images/generated_image_7.png
Saved /content/path_to_output_images/generated_image_8.png
Saved /content/path_to_output_images/generated_image_9.png
Saved /content/path_to_output_images/generated_image_10.png
Saved /content/path_to_output_images/generated_image_11.png
Saved /content/path_to_output_images/generated_image_12.png
Saved /content/path_to_output_images/generated_image_13.png
Saved /content/path_to_output_images/generated_image_14.png
Saved /content/path_to_output_images/generated_image_15.png
Saved /content/path_to_output_images/generated_image_16.png
Saved /content/path_to_output_images/generated_im

In [ ]:
import shutil
import os

# Path to the output folder with the generated images and JSON file
output_folder = '/content/path_to_output_images'  # Replace with your actual output folder path
zip_filename = '/content/output_archive.zip'  # The desired path for the zip file

# Create a zip file from the output folder
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_folder)

print(f"Zipped folder saved as: {zip_filename}")

Zipped folder saved as: /content/output_archive.zip


In [ ]:
import cv2
import os

# Path to the folder containing your icons
folder_path = '/content/images'

# List to store the icon names and their dimensions in your desired format
icon_dimensions = {}

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Only consider image files
        # Read the image using OpenCV
        img = cv2.imread(os.path.join(folder_path, filename))

        # Get the image dimensions (height, width)
        height, width = img.shape[:2]

        # Store the dimensions in the dictionary in the desired format
        icon_dimensions[filename] = (width, height)
        print(f"Image: {filename}, Width: {width}, Height: {height}")

# Save the icon dimensions to a Python-formatted file
output_file = 'icon_dimensions.py'
with open(output_file, 'w') as f:
    f.write("icon_dimensions = {\n")
    for key, value in icon_dimensions.items():
        f.write(f'    "{key}": {value},  # Width = {value[0]}, Height = {value[1]}\n')
    f.write("}\n")

print(f"Icon dimensions saved to {output_file}")


Image: pic18.png, Width: 99, Height: 36
Image: pic19.png, Width: 81, Height: 40
Image: pic10.png, Width: 162, Height: 56
Image: pic4.png, Width: 191, Height: 64
Image: pic14.png, Width: 45, Height: 40
Image: pic15.png, Width: 71, Height: 69
Image: pic3.png, Width: 88, Height: 42
Image: pic5.png, Width: 221, Height: 77
Image: pic17.png, Width: 35, Height: 31
Image: pic7.png, Width: 66, Height: 53
Image: pic6.png, Width: 60, Height: 52
Image: pic16.png, Width: 50, Height: 39
Image: pic11.png, Width: 51, Height: 68
Image: pic2.png, Width: 200, Height: 29
Image: pic1.png, Width: 155, Height: 91
Image: pic21.png, Width: 55, Height: 30
Image: pic20.png, Width: 61, Height: 34
Image: pic22.png, Width: 53, Height: 29
Image: pic9.png, Width: 174, Height: 54
Image: pic13.png, Width: 46, Height: 41
Image: pic8.png, Width: 68, Height: 58
Image: pic12.png, Width: 45, Height: 41
Icon dimensions saved to icon_dimensions.py
